# Acquire

1st step is to aquire the data from the Codeup db.

In [ ]:
#Libraries

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# preparing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, explained_variance_score

# modeling and evaluating
from sklearn.linear_model import LinearRegression, LassoLars
from sklearn.linear_model import TweedieRegressor
from sklearn.feature_selection import RFE
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.preprocessing import PolynomialFeatures

# Exploring
import scipy.stats as stats
import math

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
# pd.options.display.float_format = '{:20,.2f}'.format

# import acquire
# import summarize
# import prepare

In [ ]:
import acquire

In [ ]:
import os
import env

In [ ]:
# Add how handling nulls and document how we're handling them.
# Handle outliers and document (add to README).

In [ ]:
# Importing and displaying the zillow dataframe

df = acquire.get_zillow_data()
df.head()

In [ ]:
# Finding the shape of the dataframe to make sure it matches the shape of the data that I built in MySQLPro.

df.shape

In [ ]:
# Using .info() to find the dtypes of the columns, and to have a baseline count of non-nulls in each column to compare to my nulls when I get into the prep stage.

df.info()

In [ ]:
df.describe()

In [ ]:
df.taxamount.value_counts()

In [ ]:
# I think a better way of dividing up these variables might be using cut??
def roundup(x):
    return int(math.ceil(x / 100.0)) * 100

In [ ]:
def rounddown(x):
    return int(math.floor(x / 100.0)) * 100

3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [ ]:
df.isna().sum()

In [ ]:
df.shape

In [ ]:
num_row_missing = df.isna().sum()
num_row_missing

In [ ]:
# Decided to round this column, but can remove the round() command if needed to match the curriculum exactly.
pct_rows_missing = round(num_row_missing/df.shape[0], 3)

In [ ]:
df_missing = pd.DataFrame({'num_row_missing': num_row_missing, 'pct_rows_missing': pct_rows_missing})
df_missing.head()

In [ ]:
# Now to create the function:

def zillow_missing_values(df):
    
    total_rows = df.shape[0]
    
    # Count of missing values per column
    num_row_missing = df.isna().sum()
    
    # Pct of missing values per column
    pct_rows_missing = num_row_missing/total_rows
    
    df_missing = pd.DataFrame({'num_row_missing': num_row_missing, 'pct_rows_missing': pct_rows_missing})
    
    return df_missing
    

In [ ]:
df.head()

In [ ]:
df_missing_test = zillow_missing_values(df)
df_missing_test

#### Takeaways on Missing Values

- There are three points about taking care of the missing values:
    1. Columns at > 50% missing values should probably be dropped.
    2. Or if there is no need for the column from a modeling perspective, and most of the column is missing values, then it's dropped.
    3. Conversely, if a column has > 50% missing values but there is a modeling need for that data, the column may be retained.

### Columns Missing Info

4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [ ]:
df.nunique()

In [ ]:
# Percentage of each row that have missing column info.
df.isna().sum(axis = 1)

In [ ]:
# Percentage of each row that do not have missing column info.

# df.notna().sum(axis = 1) / df.shape[1]
df.isna().sum(axis = 1) / df.shape[1]

In [ ]:
# of rows missing

df.isna().sum(axis = 1).value_counts().sort_index()

In [ ]:
# pct cols missing
(df.isna().sum(axis = 1).value_counts(normalize = True).sort_index() * 100).reset_index()

In [ ]:
# Trying to reset the column so that I can get the dataframe above...

df_col = pd.DataFrame({'pct_cols_missing': (df.isna().sum(axis = 1).value_counts(normalize = True).sort_index() * 100).reset_index()})

In [ ]:
df_col['num_cols_missing'] = df.isna().sum(axis = 1)
df_col

In [ ]:
# This isn't right.

In [ ]:
def count_and_percent_missing_column(df):
    num_rows = df.loc[:].isnull().sum()
    num_cols_missing = df.loc[:, df.isna().any()].count()
    pct_cols_missing = round(df.loc[:, df.isna().any()].count() / len(df.index) * 100, 3)
    missing_cols_and_rows_df = pd.DataFrame({'num_cols_missing': num_cols_missing,
                                             'pct_cols_missing': pct_cols_missing,
                                             'num_rows': num_rows})
    missing_cols_and_rows_df = missing_cols_and_rows_df.fillna(0)
    missing_cols_and_rows_df['num_cols_missing'] = missing_cols_and_rows_df['num_cols_missing'].astype(int)
    return missing_cols_and_rows_df

In [ ]:
df2_test = count_and_percent_missing_column(df)
df2_test.head()

In [ ]:
df['num_cols_missing'] = df.isna().sum(axis = 1)
df.num_cols_missing.head()

In [ ]:
df['pct_cols_missing'] = df.isna().sum(axis = 1) / df.shape[1]
df.head()

In [ ]:
round(df.loc[:, df.isna().any()].count() / len(df.index) * 100, 3)

In [ ]:
df.loc[:, df.isna().any()]

In [ ]:
df.isna().sum(axis = 1) / df.shape[0]

In [ ]:
# I still think I should be able to use what I've already done above with value_counts to complete this.

In [ ]:
df.isnull()/df.shape[1]

In [ ]:
df_col = df

In [ ]:
num_missing = df.isnull().sum()
rows_count = df.shape[0]
pct_missing = num_missing / rows_count

In [ ]:
cols_missing = pd.DataFrame({'number_missing_rows': num_missing, 'pct_rows_missing': pct_missing})

In [ ]:
# Ryan's solutions:

def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows_count = df.shape[0]
    pct_missing = num_missing / rows_count
    cols_missing = pd.DataFrame({'number_missing_rows': num_missing, 'pct_rows_missing': pct_missing})
    return cols_missing

In [ ]:
nulls_by_col(df)

In [ ]:
# Summary; I need to better understand how the [] and [[]] work together to creating iteration and when they do not create iteration.

# I still think there's a way to use value_counts() to get this done, but I'm running into problems with the %

# So I was close; Ryan in his answers kinda used that concept.

In [ ]:
# Attempting to build dataframe only using .value_counts to group.
# (df.isna().sum(axis = 1).value_counts(normalize = True).sort_index() * 100).reset_index()

df3_test = pd.DataFrame(df.isna().sum(axis = 1).value_counts().sort_index().reset_index())

In [ ]:
# Ryan's solution:

def nulls_by_row(df):
    num_cols_missing = df.isnull().sum(axis=1)
    pct_cols_missing = df.isnull().sum(axis=1)/df.shape[1]*100
    rows_missing = pd.DataFrame({'num_cols_missing': num_cols_missing, 'pct_cols_missing': pct_cols_missing}).reset_index().groupby(['num_cols_missing','pct_cols_missing']).count().rename(index=str, columns={'index': 'num_rows'}).reset_index()
    return rows_missing 

null_rows = nulls_by_row(df)
null_rows.sort_values(by="pct_cols_missing", ascending=False, inplace=True)
null_rows

In [ ]:
# Nice. So the key here is that I use the .value_counts AFTER we create the function. Not inside the function.

# Prepare

1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 70)

In [ ]:
df.info()

In [ ]:
df.propertylandusedesc.value_counts()

#### Plan for Isolating Single Use properties:

Since the vast majority of all single use properties appear to be single use homes, I'm going to use these codes to isolate the properties that I want: 261.0, 260.0, 262.0, 263.0, 264.0. That way I'm not only using `unitcnt`.

In [ ]:
# Looking at the type of building and remove those that aren't highly likely to be single unit properties...

include_ids = [261.0, 260.0, 262.0, 263.0, 264.0]
df = df[df.propertylandusetypeid.isin(include_ids)]
df.shape

In [ ]:
df.unitcnt.value_counts()

## Question 2:

Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

- The input:
    - A dataframe
    - A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
    - A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
- The output:
    - The dataframe with the columns and rows dropped as indicated. *Be sure to drop the columns prior to the rows in your function.*
- **Hint:**
    - Look up the dropna documentation.
    - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
    - Make use of inplace, i.e. inplace=True/False.

In [ ]:
df.head()

In [ ]:
# Dropping the unecessary columns:

df = df.drop(columns = ['id.1', 'id'] )

In [ ]:
def drop_id_cols(df):
    df = df.drop(columns = ['id.1', 'id'])
    return df

In [ ]:
threshold = int(round(.6*len(df.index),0))
threshold

In [ ]:
len(df.index)

## Testing the property function

In [ ]:
zillow = acquire.get_zillow_data()
zillow.shape

In [ ]:
id_list_choice = [261.0, 260.0, 262.0, 263.0, 264.0]

In [ ]:
def property_type_focus(df, id_list):
#     id_list = [261.0, 260.0, 262.0, 263.0, 264.0]
    df = df[df.propertylandusetypeid.isin(id_list)]
    return df

In [ ]:
zillow.shape

In [ ]:
zillow = property_type_focus(zillow, id_list_choice)
zillow.shape

In [ ]:
def handle_missing_values(df, col_limit = .6, row_limit = .6):
    
    df.drop(columns = ['id.1', 'id', 'propertyzoningdesc', 'calculatedbathnbr'], inplace = True)
    df.drop
    # Setting the threshold for columns to drop:
    col_thresh = int(round(col_limit * len(df.index), 0))
    df.dropna(axis = 1, thresh = col_thresh, inplace = True)
    # Now for the rows:
    row_thresh = int(round(col_limit * len(df.columns), 0))
    df.dropna(axis = 0, thresh = row_thresh, inplace = True)
    return df
    

In [ ]:
zillow = handle_missing_values(zillow)

In [ ]:
zillow.shape

## Splitting Zillow Data Function

In [ ]:
def split_zillow_data(df):
    # df = get_mall_data()
    # Splitting my data based on the target variable of tenure:
    train_validate, test = train_test_split(df, test_size=.15, random_state=123)
    
    # Splitting the train_validate set into the separate train and validate datasets.
    train, validate = train_test_split(train_validate, test_size=.20, random_state=123)
    
    # Printing the shape of each dataframe:
    print(f'Shape of train df: {train.shape}')
    print(f'Shape of validate df: {validate.shape}')
    print(f'Shape of test df: {test.shape}')
    return train, validate, test

In [ ]:
train, validate, test = split_zillow_data(zillow)
train.head()

In [ ]:
train.isnull().sum().sort_values(ascending = False)

### 3. Filling Missing Values

**After splitting the df into train, validate, test:**

Decide how to handle the remaining missing values:

- Fill with constant value.
- Impute with mean, median, mode.
- Drop row/column

In deciding how to fill the missing values, I'm going to for the mean for the majority of the columns. Knowing the data and what I'm trying to find from it, adding a bunch of values to the average *shouldn't* change the outcome of any model I'm building too much. However, I'll have to bookmark the decision I'm making here and come back to it (and maybe change my mind) if the models I start creating are appears to be messed up in some way.

In [ ]:
# Impute missing values, after splitting.

cols = [
    "structuretaxvaluedollarcnt",
    "taxamount",
    "taxvaluedollarcnt",
    "landtaxvaluedollarcnt",
    "structuretaxvaluedollarcnt",
    "finishedsquarefeet12",
    "calculatedfinishedsquarefeet",
    "fullbathcnt",
    "lotsizesquarefeet",
    "heatingorsystemtypeid"
]


for col in cols:
    median = train[col].median()
    train[col].fillna(median, inplace=True)
    validate[col].fillna(median, inplace=True)
    test[col].fillna(median, inplace=True)

In [ ]:
train.isnull().sum().sort_values(ascending = False)

In [ ]:
# Categorical/Discrete columns to use mode to replace nulls

cols = [
    "buildingqualitytypeid",
    "regionidcity",
    "regionidzip",
    "yearbuilt",
    "regionidcity",
    "censustractandblock"
]

for col in cols:
    mode = int(train[col].mode()) # I had some friction when this returned a float (and there were no decimals anyways)
    train[col].fillna(value=mode, inplace=True)
    validate[col].fillna(value=mode, inplace=True)
    test[col].fillna(value=mode, inplace=True)

In [ ]:
train.isnull().sum().sort_values(ascending = False)

In [ ]:
cols3 = [
    "unitcnt"
]

for col in cols3:
    train[col].fillna(value=1, inplace=True)
    validate[col].fillna(value=1, inplace=True)
    test[col].fillna(value=1, inplace=True)

In [ ]:
train.isnull().sum().sort_values(ascending = False)

In [ ]:
def imputing_missing_values(train, validate, test):
    
    
    cols = [
    "structuretaxvaluedollarcnt",
    "taxamount",
    "taxvaluedollarcnt",
    "landtaxvaluedollarcnt",
    "structuretaxvaluedollarcnt",
    "finishedsquarefeet12",
    "calculatedfinishedsquarefeet",
    "fullbathcnt",
    "lotsizesquarefeet",
    "heatingorsystemtypeid"
    ]


    for col in cols:
        median = train[col].median()
        train[col].fillna(median, inplace=True)
        validate[col].fillna(median, inplace=True)
        test[col].fillna(median, inplace=True)


    # Categorical/Discrete columns to use mode to replace nulls

    cols2 = [
        "buildingqualitytypeid",
        "regionidcity",
        "regionidzip",
        "yearbuilt",
        "regionidcity",
        "censustractandblock"
    ]

    for col in cols2:
        mode = int(train[col].mode()) # I had some friction when this returned a float (and there were no decimals anyways)
        train[col].fillna(value=mode, inplace=True)
        validate[col].fillna(value=mode, inplace=True)
        test[col].fillna(value=mode, inplace=True)
    
    # Taking care of unit count.
    cols3 = [
        "unitcnt"
    ]

    for col in cols3:
        train[col].fillna(value=1, inplace=True)
        validate[col].fillna(value=1, inplace=True)
        test[col].fillna(value=1, inplace=True)
    
        
    return train, validate, test

In [ ]:
train.shape

In [ ]:
train.isnull().sum()

In [ ]:
def imputing_missing_values(df):
    
    # First, inputing the median values:
    df.regionidcity = df.regionidcity.fillna(df.regionidcity.median())
    df.regionidzip = df.regionidzip.fillna(df.regionidzip.median())
    df.yearbuilt = df.yearbuilt.fillna(df.yearbuilt.median())
    df.censustractandblock = df.censustractandblock.fillna(df.censustractandblock.median())
    
    # Now using the mean to input the rest of the missing values:
    df.lotsizesquarefeet = df.lotsizesquarefeet.fillna(df.lotsizesquarefeet.mean())
    df.finishedsquarefeet12 = df.finishedsquarefeet12.fillna(df.finishedsquarefeet12.mean())
    df.calculatedbathnbr = df.calculatedbathnbr.fillna(df.calculatedbathnbr.mean())
    df.fullbathcnt = df.fullbathcnt.fillna(round(df.fullbathcnt.mean(),0))
    df.calculatedfinishedsquarefeet = df.calculatedfinishedsquarefeet.fillna(round(df.calculatedfinishedsquarefeet.mean(),0))
    df.structuretaxvaluedollarcnt = df.structuretaxvaluedollarcnt.fillna(round(df.structuretaxvaluedollarcnt.mean(),0))
    df.taxamount = df.taxamount.fillna(round(df.taxamount.mean(),0))
    df.landtaxvaluedollarcnt = df.landtaxvaluedollarcnt.fillna(round(df.landtaxvaluedollarcnt.mean(),0))
    df.taxvaluedollarcnt = df.taxvaluedollarcnt.fillna(round(df.taxvaluedollarcnt.mean(),0))
    

## Now imputing values

In [ ]:
# I should actually be imputing values AFTER I split the data.

median = df.regionidcity.median()
df.regionidcity = df.regionidcity.fillna(df.regionidcity.median())
df.regionidzip = df.regionidzip.fillna(df.regionidzip.median())
df.censustractandblock = df.censustractandblock.fillna(df.censustractandblock.median())

In [ ]:
df.isna().sum().sort_values(ascending = False)

In [ ]:
# Now for the mean:

df.lotsizesquarefeet = df.lotsizesquarefeet.fillna(df.lotsizesquarefeet.mean())
df.finishedsquarefeet12 = df.finishedsquarefeet12.fillna(df.finishedsquarefeet12.mean())
df.calculatedbathnbr = df.calculatedbathnbr.fillna(df.calculatedbathnbr.mean())
df.fullbathcnt = df.fullbathcnt.fillna(round(df.fullbathcnt.mean(),0))
df.calculatedfinishedsquarefeet = df.calculatedfinishedsquarefeet.fillna(round(df.calculatedfinishedsquarefeet.mean(),0))
df.structuretaxvaluedollarcnt = df.structuretaxvaluedollarcnt.fillna(round(df.structuretaxvaluedollarcnt.mean(),0))
df.taxamount = df.taxamount.fillna(round(df.taxamount.mean(),0))
df.landtaxvaluedollarcnt = df.landtaxvaluedollarcnt.fillna(round(df.landtaxvaluedollarcnt.mean(),0))
df.taxvaluedollarcnt = df.taxvaluedollarcnt.fillna(round(df.taxvaluedollarcnt.mean(),0))

df.isna().sum().sort_values(ascending = False)



All Missing values removed or imputed.

In [ ]:
cols3 = ["heatingorsystemdesc"]

for col in cols3:
#     median = train[col].median()
    train[col].fillna("None", inplace = True)
    validate[col].fillna("None", inplace = True)
    test[col].fillna("None", inplace = True)

In [ ]:
df.head()

# Mall Customers

**notebook**
- Acquire data from mall_customers.customers in mysql database.
- Summarize data (include distributions and descriptive statistics).
- Detect outliers using IQR.
- Split data (train-test-split).
- Encode categorical columns using a one hot encoder.
- Handles missing values.
- Scaling

wrangle_mall.py
- Acquire data from mall_customers.customers in mysql database.
- Split the data
- One-hot-encoding
- Missing values
- Scaling

In [ ]:
import mall_wrangle

In [ ]:
df_mall = mall_wrangle.get_mall_data()
df_mall.head()

In [ ]:
# Summarizing the data:

df_mall.info()

In [ ]:
print(f'The shape of this dataframe is {df_mall.shape}')
df_mall.describe()

### Detect outliers using IQR.

In [ ]:
# Visualizing the distribution of the columns

df_mall.hist(figsize=(24, 10), bins = 20)
plt.show()

In [ ]:
df_mall.spending_score.hist(bins = 25)

In [ ]:
q1 = df_mall.spending_score.quantile(.25)
q3 = df_mall.spending_score.quantile(.75)

q1, q3
iqr = q3 - q1

In [ ]:
k = 1.5
upper_bound = q3 + (k * iqr)
upper_bound

In [ ]:
df_mall[df_mall.spending_score > upper_bound]

In [ ]:
lower_bound = q1 - (k * iqr)
lower_bound
df_mall[df_mall.spending_score < lower_bound]

Does not appear to be any observations outside of the upper or lower bounds.

In [ ]:
def get_upper_outliers(s, k):
    '''
    Given a series and a cutoff value, k, returns the upper outliers for the
    series.

    The values returned will be either 0 (if the point is not an outlier), or a
    number that indicates how far away from the upper bound the observation is.
    '''
    q1, q3 = s.quantile([.25, .75])
    iqr = q3 - q1
    upper_bound = q3 + k * iqr
    return s.apply(lambda x: max([x - upper_bound, 0]))

def add_upper_outlier_columns(df, k):
    '''
    Add a column with the suffix _outliers for all the numeric columns
    in the given dataframe.
    '''
    # outlier_cols = {col + '_outliers': get_upper_outliers(df[col], k)
    #                 for col in df.select_dtypes('number')}
    # return df.assign(**outlier_cols)

    for col in df.select_dtypes('number'):
        df[col + '_outliers'] = get_upper_outliers(df[col], k)

    return df

add_upper_outlier_columns(df_mall, k=1.5)

df_mall.head()

### Split the data

In [ ]:
def split_mall_data_test(df):
    # df = get_mall_data()
    # Splitting my data based on the target variable of tenure:
    train_validate, test = train_test_split(df, test_size=.15, random_state=123)
    
    # Splitting the train_validate set into the separate train and validate datasets.
    train, validate = train_test_split(train_validate, test_size=.20, random_state=123)
    
    # Printing the shape of each dataframe:
    print(f'Shape of train df: {train.shape}')
    print(f'Shape of validate df: {validate.shape}')
    print(f'Shape of test df: {test.shape}')
    return train, validate, test

In [ ]:
# I built a function previously that split the mall data:

train, validate, test = split_mall_data(df_mall)
train.head()

### Encode categorical columns using a one hot encoder.

In [ ]:
# Based on what I've read, .get_dumbies is a one-hot-encoder. It's only a true dummy encoder if I select to drop the first column in the function.

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False, categories='gender')

In [ ]:
# Couldn't get this to work...
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False, categories='gender')
# train_matrix = ohe.fit_transform(train[['col']])
# validate_matrix = ohe.transform(validate[['col']])
# test_matrix = ohe.transform(test[['col']])
# train.head()

In [ ]:
train['is_male'] = pd.get_dummies(train.gender, drop_first = True) # Leaving the drop_first() argument set to True means that both columns are return.
train.head()
# One for female, one for male. Note that returns an "extra" column that isn't strictly necessary.

In [ ]:
validate['is_male'] = pd.get_dummies(validate.gender, drop_first = True)
test['is_male'] = pd.get_dummies(test.gender, drop_first = True)

validate.shape, test.shape

In [ ]:
# If I run the OneHotEncoder, I should end up with basically the same result.

enc = preprocessing.OneHotEncoder()

enc.fit(train)

In [ ]:
onehotlabels = enc.transform(train).toarray(1)
onehotlabels.shape

In [ ]:
onehotlabels

I couldn't get the OneHotEncoder to work. Still, I was able to accomplish the work by using the .get_dummies, and simply not setting the drop_first argument = False.

#### Handles missing values.

In [ ]:
# There do not appear to be any misisng values, but I'll add some code in there...

train.isnull().sum()

In [ ]:
# For filling in any nulls
train.spending_score.fillna(train.spending_score.mean(), inplace = True)
train.age.fillna(train.age.mean(), inplace = True)
train.annual_income.fillna(train.annual_income.median(), inplace = True)


### Scaling 

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train.head()